In [5]:
import pandas as pd

# Mostrar todo el contenido de cada celda (sin recortes)
pd.set_option("display.max_colwidth", None)

We check to see if we can find the robots.txt file to determine whether web scraping is allowed.


revisamos a ver si encontramos el archivo robots.txt para saber si esta permitido realizar el web scrapy

In [ ]:
import urllib.robotparser

url = "https://rentabien.com.co/inmuebles/"

rp = urllib.robotparser.RobotFileParser()
rp.set_url(url + "/robots.txt")
rp.read()

user_agent = "*"
pagina = url + "/ruta-que-quieres-scrapear"

if rp.can_fetch(user_agent, pagina):
    print("Está permitido hacer scraping en esta ruta")
else:
    print("NO está permitido hacer scraping en esta ruta")


Está permitido hacer scraping en esta ruta


Now we look for the file where the terms and conditions are located.



ahora buscamos el archivo donde esta termino y condicciones 

In [4]:
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://example.com"

posibles_rutas = [
    "/terms",
    "/terms-of-service",
    "/tos",
    "/legal",
    "/condiciones",
    "/terminos"
]

headers = {
    "User-Agent": "Mozilla/5.0 (compatible; MiBot/1.0)"
}

def buscar_terminos():
    for ruta in posibles_rutas:
        url = BASE_URL + ruta
        try:
            response = requests.get(url, headers=headers, timeout=5)
            if response.status_code == 200:
                print(f"Página encontrada: {url}")
                return response.text
        except:
            pass
    print("No se encontró página de términos automáticamente")
    return None

def analizar_contenido(html):
    soup = BeautifulSoup(html, "html.parser")
    texto = soup.get_text().lower()

    palabras_clave = [
        "scraping",
        "data mining",
        "automat",
        "bot",
        "crawler",
        "robot"
    ]

    encontradas = [p for p in palabras_clave if p in texto]

    if encontradas:
        print("⚠️ Posibles restricciones encontradas:")
        for palabra in encontradas:
            print(" -", palabra)
    else:
        print("No se encontraron menciones directas a scraping")

html = buscar_terminos()

if html:
    analizar_contenido(html)


No se encontró página de términos automáticamente


We proceed with extracting the property links for their subsequent data extraction.


procedemos con la extraccion de los link de los inmuebles para su posterior extraccion de datos

In [ ]:
 #etapa_1_guardar_enlaces.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Configurar navegador
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=chrome_options)

url = "https://rentabien.com.co/inmuebles/"
driver.get(url)

WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "properties")))
time.sleep(3)

all_links = []
page_number = 1

while True:
    print(f"\n🟨 Página {page_number} cargada")

    # Esperar los inmuebles visibles
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='properties']//article")))
    except:
        print("No se encontraron inmuebles. Fin.")
        break

    # Extraer links
    inmuebles = driver.find_elements(By.XPATH, "//*[@id='properties']//article")
    for inm in inmuebles:
        try:
            link = inm.find_element(By.XPATH, ".//a[contains(@href, 'detalle')]").get_attribute("href")
            all_links.append(link)
        except:
            pass
    print(f"  ➕ Total acumulado: {len(all_links)} enlaces")

    # Intentar pasar a la siguiente página
    try:
        current = driver.find_element(By.CSS_SELECTOR, "a.page-numbers.current")
        next_page = str(int(current.text) + 1)

        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, f"//a[@class='page-numbers' and text()='{next_page}']"))
        )

        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)
        next_button.click()

        # Esperar que cambie contenido (nuevo primer inmueble)
        WebDriverWait(driver, 10).until(
            EC.staleness_of(inmuebles[0])
        )
        time.sleep(3)
        page_number += 1

    except:
        print("✅ No hay más páginas. Finalizando recolección de enlaces.")
        break

# Guardar los enlaces en un archivo CSV
with open("enlaces_inmuebles.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    for link in all_links:
        writer.writerow([link])

print(f"\n✅ Total enlaces guardados: {len(all_links)} en 'enlaces_inmuebles.csv'")
driver.quit()



We proceed to extract the relevant data, but for privacy reasons we leave out the code and URL fields, since they are not useful for the analysis and excluding them is even good practice for the ML process. During the extraction, we also apply some data cleaning before saving the dataset.




procedemos a extraer los datos releveante pero por mayor privacidad dejamos fuera los datos de codigo y de URL el cual no es util en el analisis y para el proceso de ML es incluso una buena practica en la extraccion tambien aplicamos un poco de limpieza de datos antes de guardar

In [ ]:
# etapa_2_extraer_datos.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import csv
from datetime import datetime
import re

# Leer enlaces
with open("enlaces_inmuebles.csv", "r", encoding="utf-8") as f:
    enlaces = [line.strip() for line in f if line.strip()]

print(f"🔗 Total enlaces leídos: {len(enlaces)}")

# Configurar navegador en modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)

# Campos para el CSV (SIN Código ni URL)
campos = ["Tipo", "Operación", "Precio", "Administración", "Niveles", "Habitaciones", "Baños",
          "Parqueaderos", "Área Construida", "Área Lote", "Barrio", "Estrato", "Ciudad"]

def extraer_datos(url):
    driver.get(url)
    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.ID, "domus_property"))
        )
    except TimeoutException:
        print(f"⚠️ Primer intento fallido, reintentando: {url}")
        driver.refresh()
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.ID, "domus_property"))
            )
        except TimeoutException:
            print(f"❌ Falló definitivamente: {url}")
            return {k: "N/A" for k in campos}

    time.sleep(1)

    # Diccionario SIN URL
    datos = {k: "N/A" for k in campos}

    # Extraer tipo y operación
    try:
        titulo = driver.find_element(By.CSS_SELECTOR, "h3.entry-title").text.strip()
        partes = titulo.split(" en ")
        if len(partes) == 2:
            datos["Tipo"] = partes[0].strip()
            datos["Operación"] = partes[1].strip()
    except:
        pass

    # Precio
    try:
        datos["Precio"] = driver.find_element(
            By.XPATH,
            "//*[@id='domus_property']//div/div[2]/div/div[2]/h5"
        ).text.strip()
    except:
        pass

    # Administración
    try:
        admin = driver.find_element(
            By.XPATH,
            "//*[@id='domus_property']//div/div[2]/div/div[2]/p"
        ).text.strip()
        datos["Administración"] = admin.replace("Administración: ", "")
    except:
        pass

    # Características numéricas
    xpaths = {
        "Niveles": 1,
        "Habitaciones": 2,
        "Baños": 3,
        "Parqueaderos": 4,
        "Área Construida": 5,
        "Área Lote": 6,
    }

    for campo, i in xpaths.items():
        try:
            texto = driver.find_element(
                By.XPATH,
                f"//*[@id='domus_property']//div/div[3]/div/div[{i}]/div/div[2]"
            ).text.strip()
            datos[campo] = texto
        except:
            pass

    # Barrio (SIN Código)
    try:
        bloque = driver.find_element(
            By.XPATH,
            "//*[@id='domus_property']//div[@class='col-md-6']"
        ).get_attribute("innerHTML")
        barrio_match = re.search(r'Barrio:</b>\s*([^<]+)', bloque)
        if barrio_match:
            datos["Barrio"] = barrio_match.group(1).strip()
    except:
        pass

    # Estrato y Ciudad
    try:
        texto = driver.find_element(
            By.XPATH,
            "//*[@id='domus_property']//div/div[5]/div[2]"
        ).text
        for linea in texto.split("\n"):
            if "Estrato:" in linea:
                datos["Estrato"] = linea.replace("Estrato:", "").strip()
            elif "Ciudad:" in linea:
                datos["Ciudad"] = linea.replace("Ciudad:", "").strip()
    except:
        pass

    return datos


def limpiar_numero(texto):
    if texto == "N/A":
        return texto
    match = re.search(r"\d+[\.,]?\d*", texto.replace(".", "").replace(",", "."))
    return match.group(0) if match else "N/A"


def limpiar_valor_pesos(texto):
    if texto == "N/A":
        return texto
    return re.sub(r"[^\d]", "", texto)


datos_finales = []

for idx, enlace in enumerate(enlaces, 1):
    print(f"🔎 ({idx}/{len(enlaces)}) {enlace}")
    datos = extraer_datos(enlace)

    datos["Precio"] = limpiar_valor_pesos(datos["Precio"])
    datos["Administración"] = limpiar_valor_pesos(datos["Administración"])

    for campo in [
        "Niveles",
        "Habitaciones",
        "Baños",
        "Parqueaderos",
        "Área Construida",
        "Área Lote",
    ]:
        datos[campo] = limpiar_numero(datos[campo])

    datos_finales.append(datos)

# Guardar CSV final (SIN Código ni URL)
nombre_csv = f"inmuebles_detallado_{datetime.now().strftime('%Y%m%d')}.csv"
with open(nombre_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=campos)
    writer.writeheader()
    writer.writerows(datos_finales)

print(f"\n✅ Extracción completada. Guardado en: {nombre_csv}")
driver.quit()


🔗 Total enlaces leídos: 1332
🔎 (1/1332) https://rentabien.com.co/detalle?code=5134&alt=3520985
🔎 (2/1332) https://rentabien.com.co/detalle?code=14964&alt=3520852
🔎 (3/1332) https://rentabien.com.co/detalle?code=7593&alt=3520815
🔎 (4/1332) https://rentabien.com.co/detalle?code=25326&alt=3520739
🔎 (5/1332) https://rentabien.com.co/detalle?code=7592&alt=3520677
🔎 (6/1332) https://rentabien.com.co/detalle?code=25325&alt=3520619
🔎 (7/1332) https://rentabien.com.co/detalle?code=3226&alt=3520049
🔎 (8/1332) https://rentabien.com.co/detalle?code=25324&alt=3519403
🔎 (9/1332) https://rentabien.com.co/detalle?code=3225&alt=3519379
🔎 (10/1332) https://rentabien.com.co/detalle?code=7591&alt=3519257
🔎 (11/1332) https://rentabien.com.co/detalle?code=4253&alt=3519085
🔎 (12/1332) https://rentabien.com.co/detalle?code=25323&alt=3519016
🔎 (13/1332) https://rentabien.com.co/detalle?code=14963&alt=3518804
🔎 (14/1332) https://rentabien.com.co/detalle?code=14962&alt=3518684
🔎 (15/1332) https://rentabien.com.c